In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sqlite3
import numpy as np
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import openslide
import time
import pickle
import cv2

In [4]:
import matplotlib.pyplot as plt

In [5]:
from helper.nms_center_distance import non_max_suppression_by_distance

In [6]:
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
from Evaluator import *
from utils import *

In [7]:
path = Path('/data/Datasets/EIPH_WSI/')

In [8]:
database = Database()
database.open(str(path/'EIPH.sqlite'))

True

In [9]:
files = []
lbl_bbox = []

getslides = """SELECT uid, filename FROM Slides"""
for currslide, filename in tqdm(database.execute(getslides).fetchall()):
    database.loadIntoMemory(currslide)

    check = True if 'erliner' in filename else False
    slidetype = 'Berliner Blau/' if check else 'Turnbull Blue/'

    slide_path = path / slidetype / filename

    down_factor = 1

    classes = {3: 0, 4: 1, 5: 2, 6: 3, 7: 4}#{3: 1, 4: 2, 5: 3, 6: 4, 7: 5} #
    labels, bboxes = [], []
    for id, annotation in database.annotations.items():
        if annotation.labels[0].classId in classes:
            d = 2 * annotation.r / down_factor
            x_min = (annotation.x1 - annotation.r) / down_factor
            y_min = (annotation.y1 - annotation.r) / down_factor
            x_max = x_min + d
            y_max = y_min + d
            label = classes[annotation.labels[0].classId]

            bboxes.append([int(x_min), int(y_min), int(x_max), int(y_max)])
            labels.append(label)

    if len(bboxes) > 0:
        lbl_bbox.append([bboxes, labels])
        files.append(slide_path)

img2bbox = dict(zip(files, np.array(lbl_bbox)))
get_y_func = lambda o:img2bbox[o]

  0%|          | 0/24 [00:00<?, ?it/s]

Loading DB into memory ...


  4%|▍         | 1/24 [00:00<00:06,  3.35it/s]

Loading DB into memory ...


  8%|▊         | 2/24 [00:01<00:13,  1.59it/s]

...

Loading DB into memory ...
Loading DB into memory ...
Loading DB into memory ...
Loading DB into memory ...


 88%|████████▊ | 21/24 [00:05<00:00,  4.78it/s]

Loading DB into memory ...
Loading DB into memory ...


100%|██████████| 24/24 [00:07<00:00,  3.38it/s]

Loading DB into memory ...


In [10]:
def calc_ap(annotations, boxes_gt, labels_gt, file:str="temp.png"):
    boundingBoxes = BoundingBoxes()
    evaluator = Evaluator()

    boxes_pred = annotations[:, [0, 1, 2, 3]]
    # change pred from x,y,x2,y2 -> x,y,w,h
    boxes_pred[:, 2:] = boxes_pred[:, 2:] - boxes_pred[:, :2]
    labels_pred = annotations[:, 4]
    score_pred = annotations[:, 5]


    for box, cla in zip(boxes_gt, labels_gt):
        temp = BoundingBox(imageName=str(file), classId=cla, x=box[0],
                                       y=box[1],
                                       w=box[2], h=box[3], typeCoordinates=CoordinatesType.Absolute,
                                       bbType=BBType.GroundTruth, format=BBFormat.XYWH, imgSize=(5000, 5000))
        boundingBoxes.addBoundingBox(temp)

    for box, cla, scor in zip(boxes_pred, labels_pred, score_pred):
        temp = BoundingBox(imageName=str(file), classId=cla, x=box[0],
                                       y=box[1], classConfidence=scor,
                                       w=box[2], h=box[3], typeCoordinates=CoordinatesType.Absolute,
                                       bbType=BBType.Detected, format=BBFormat.XYWH, imgSize=(5000, 5000))
        boundingBoxes.addBoundingBox(temp)

    metricsPerClass = evaluator.GetPascalVOCMetrics(boundingBoxes, iou_threshold)
    return np.mean([mc['AP'] for mc in metricsPerClass])

In [11]:
resultsArchive = pickle.load(open('inference_results_boxes.p','rb'))

In [17]:
iou_threshold = 0.25

In [18]:
result_dict = {}

for file in tqdm(files):
    boxes_gt, labels_gt = get_y_func(file)
    boxes_gt, labels_gt = np.array(boxes_gt), np.array(labels_gt)
    boxes_gt[:, 2:] = boxes_gt[:, 2:] - boxes_gt[:, :2]
    
    annotations = np.array(resultsArchive[file.stem+".svs"])
    
    annotations_nms_dist = non_max_suppression_by_distance(annotations, annotations[:, 5], radius=50)
    ap = calc_ap(annotations_nms_dist, boxes_gt, labels_gt)
    
    result_dict[file] = ap

100%|██████████| 17/17 [08:12<00:00, 34.90s/it]


In [20]:
result_dict

{PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/01_EIPH_563479 Berliner Blau.svs'): 0.6458146650357519,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/02_EIPH_574162 berliner blau.svs'): 0.5010732987350869,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/03_EIPH_566933 R Berliner Blau neu.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/04_EIPH_567017 Berliner Blau.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/05_EIPH_569923 Berliner Blau.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/27_EIPH_571557 berliner blau.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/31_EIPH_568354 Berliner Blau.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/11_EIPH_575697 Berliner Blau.svs'): 0.830996757399306,
 PosixPath('/data/Datasets/EIPH_WSI/Turnbull Blue/29_EIPH_566481 L Turnbull blue.svs'): 0.9003602643718672,
 PosixPath('/data/Datasets/EIPH_WSI/Turnbull Blue/28_EIPH_569948 L Turnbull blue.svs'): nan,
 PosixPath('/data/Dat

In [23]:
iou_threshold = 0.5
result_dict_05 = {}

for file in tqdm(files):
    boxes_gt, labels_gt = get_y_func(file)
    boxes_gt, labels_gt = np.array(boxes_gt), np.array(labels_gt)
    boxes_gt[:, 2:] = boxes_gt[:, 2:] - boxes_gt[:, :2]
    
    annotations = np.array(resultsArchive[file.stem+".svs"])
    
    annotations_nms_dist = non_max_suppression_by_distance(annotations, annotations[:, 5], radius=50)
    ap = calc_ap(annotations_nms_dist, boxes_gt, labels_gt)
    
    result_dict_05[file] = ap

100%|██████████| 17/17 [08:14<00:00, 35.14s/it]


In [24]:
result_dict_05

{PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/01_EIPH_563479 Berliner Blau.svs'): 0.5850261231296516,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/02_EIPH_574162 berliner blau.svs'): 0.46807519040943024,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/03_EIPH_566933 R Berliner Blau neu.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/04_EIPH_567017 Berliner Blau.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/05_EIPH_569923 Berliner Blau.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/27_EIPH_571557 berliner blau.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/31_EIPH_568354 Berliner Blau.svs'): nan,
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/11_EIPH_575697 Berliner Blau.svs'): 0.7169832699389385,
 PosixPath('/data/Datasets/EIPH_WSI/Turnbull Blue/29_EIPH_566481 L Turnbull blue.svs'): 0.7802138092851992,
 PosixPath('/data/Datasets/EIPH_WSI/Turnbull Blue/28_EIPH_569948 L Turnbull blue.svs'): nan,
 PosixPath('/data/D

In [25]:
list(result_dict_05.values())

[0.5850261231296516,
 0.46807519040943024,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.7169832699389385,
 0.7802138092851992,
 nan,
 0.6651627909352531,
 0.6549598649101199,
 nan,
 0.6317499973224452,
 nan,
 0.7317177065426254,
 0.5271092788774118]

In [22]:
list(result_dict.values())

[0.6458146650357519,
 0.5010732987350869,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.830996757399306,
 0.9003602643718672,
 nan,
 0.7268287248457559,
 0.6993319052203227,
 nan,
 0.7064646311732796,
 nan,
 0.7643262955698891,
 0.5885902054038523]